In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import warnings
import regex as re

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
cal = pd.read_csv('../madrid/calendar/calendar_limpio.csv')
lis = pd.read_csv('../madrid/listings_detailed/listings_limpio.csv', low_memory=False)
ngdf = pd.read_csv('../madrid/ngdf_limpio.csv')

###### 1. Listings

###### 1.1. Latitude y Longitude a coordenadas cartesianas.

In [3]:
lis['x']=np.cos(lis.latitude)*np.cos(lis.longitude)
lis['y']=np.cos(lis.latitude)*np.sin(lis.longitude)
lis['z']=np.sin(lis.latitude)

lis.head()

,id,host_id,host_response_time,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,require_guest_profile_picture,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,x,y,z
0,6369,13660,within a few hours,0,1,0,Hispanoamérica,Chamartín,40.45628,-3.67763,Apartment,Private room,2,1,1,0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Elevator,Heat...",70,0,5,2,15,1,365,14,23,53,53,65,6,98,10,10,9,10,9,10,f,1,0,1,0,0,0.796991,-0.473457,0.375026
1,21853,83531,unknown,0,1,1,Cármenes,Latina,40.40341,-3.74084,Apartment,Private room,1,1,1,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",17,0,0,1,8,4,40,0,0,0,48,33,0,92,9,9,10,10,8,9,f,2,0,2,0,0,0.748057,-0.510947,0.423490
2,24805,101471,within a day,0,1,1,Universidad,Centro,40.42202,-3.70395,Apartment,Entire home/apt,3,1,0,1,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,E...",80,200,30,2,0,5,730,19,49,79,354,2,0,100,8,8,10,10,10,10,f,1,1,0,0,0,0.772927,-0.487128,0.406559
3,24836,101653,within a few hours,1,1,0,Justicia,Centro,40.41995,-3.69764,Apartment,Entire home/apt,4,1,2,3,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",115,200,0,4,0,3,1125,3,18,48,301,64,18,98,10,10,10,10,10,10,f,1,1,0,0,0,0.775269,-0.481796,0.408449
4,26825,114340,within an hour,0,1,1,Legazpi,Arganzuela,40.38985,-3.69011,House,Private room,1,1,1,1,Real Bed,"{Wifi,""Wheelchair accessible"",Doorman,Elevator...",25,0,15,1,0,2,365,13,40,70,337,136,27,94,10,10,10,10,10,10,f,1,0,1,0,1,0.768033,-0.469320,0.435735


###### 1.2. Property_type.

In [4]:
lis['property_type'].value_counts()

Apartment                 16897
House                      1183
Loft                        740
Condominium                 670
Serviced apartment          391
Hostel                      174
Bed and breakfast           142
Boutique hotel               93
Guest suite                  85
Other                        73
Townhouse                    60
Chalet                       54
Guesthouse                   47
Hotel                        46
Casa particular (Cuba)       32
Tiny house                   20
Camper/RV                    16
Villa                         8
Dome house                    7
Aparthotel                    7
Hut                           2
Pension (South Korea)         2
Earth house                   2
Farm stay                     1
Barn                          1
Tent                          1
Bungalow                      1
Castle                        1
Nature lodge                  1
Igloo                         1
Name: property_type, dtype: int64

In [5]:
lis['property_type'] = lis['property_type'].str.replace(r'.*Serviced apartment.*', 'Apartment', regex=True)\
                                             .str.replace(r'.*Bed and breakfast.*', 'Hotel', regex=True)\
                                             .str.replace(r'.*Hut.*', 'Other', regex=True)\
                                             .str.replace(r'.*Pension.*', 'Other', regex=True)\
                                             .str.replace(r'.*Earth house.*', 'Other', regex=True)\
                                             .str.replace(r'.*Farm stay.*', 'Other', regex=True)\
                                             .str.replace(r'.*Barn.*', 'Other', regex=True)\
                                             .str.replace(r'.*Tent.*', 'Other', regex=True)\
                                             .str.replace(r'.*Bungalow.*', 'Other', regex=True)\
                                             .str.replace(r'.*Castle.*', 'Other', regex=True)\
                                             .str.replace(r'.*Nature lodge.*', 'Other', regex=True)\
                                             .str.replace(r'.*Igloo.*', 'Other', regex=True)\
                                             .str.replace(r'.*Casa particular (Cuba).*', 'Casa', regex=True)

lis['property_type'].value_counts()


In [6]:
property_type=['property_type']

lis=pd.get_dummies(lis, columns=property_type, drop_first=True)

###### 1.3. room_type.

In [7]:
lis.room_type.unique()

array(['Private room', 'Entire home/apt', 'Hotel room', 'Shared room'],
      dtype=object)

In [8]:
room_type=['room_type']

lis=pd.get_dummies(lis, columns=room_type, drop_first=True)

###### 1.4. bed_type

In [9]:
lis. bed_type.unique()

array(['Real Bed', 'Pull-out Sofa', 'Futon', 'Couch', 'Airbed'],
      dtype=object)

In [10]:
bed_type=['bed_type']

lis=pd.get_dummies(lis, columns=bed_type, drop_first=True)

###### 1.5. amenities

In [11]:
lis. amenities.unique()

array(['{Wifi,"Air conditioning",Kitchen,Elevator,Heating,"Family/kid friendly",Washer,Essentials,Shampoo,Hangers,"Hair dryer",Iron,"Hot water","Bed linens","Extra pillows and blankets","Pocket wifi"}',
       '{TV,Internet,Wifi,"Air conditioning",Kitchen,"Free parking on premises",Doorman,Elevator,Heating,Washer,"First aid kit","Fire extinguisher",Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","translation missing: en.hosting_amenity_49","translation missing: en.hosting_amenity_50","Hot water","Bed linens","Extra pillows and blankets","Pocket wifi",Microwave,"Coffee maker",Refrigerator,"Dishes and silverware","Cooking basics",Oven}',
       '{TV,Internet,Wifi,"Air conditioning",Kitchen,Elevator,"Buzzer/wireless intercom",Heating,"Family/kid friendly",Washer,"Fire extinguisher",Essentials,Shampoo,"24-hour check-in",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","Pack ’n Play/travel crib","Hot water",Microwave,"Coffee maker",Refrig

In [12]:
#amenities_full = set()
#for amenities_list in lis.amenities.str.strip("{}").str.replace('"', '').str.split(','):
    #amenities_full.update([amenity.strip() for amenity in amenities_list if amenity.strip() != '' and 'translation missing' not in amenity])


In [13]:
amenities_full = set()
for amenities_list in lis.amenities.str.strip("{}").str.replace('"', '').str.split(','):
    amenities_full.update([amenity.strip() for amenity in amenities_list if amenity.strip() != '' and 'translation missing' not in amenity])

amenities_m = []
for fila in lis.amenities.str.strip("{}").str.replace('"', '').str.split(','):
    row = []
    for servicio in amenities_full:
        if servicio in fila:
            row.append(1)
        else:
            row.append(0)
    amenities_m.append(row)

In [14]:
amenities = pd.DataFrame(amenities_m, columns=list(amenities_full))

In [15]:
amenities.head()

,Breakfast table,Indoor fireplace,Convection oven,Soaking tub,Extra space around bed,Essentials,Pets allowed,Electric profiling bed,Heated floors,Bidet,Beach view,Breakfast,Safety card,Fire extinguisher,24-hour check-in,Smart lock,HBO GO,Shampoo,EV charger,Wide entrance for guests,Children’s dinnerware,Lockbox,Smart TV,Wine cooler,High chair,Bathtub,Double oven,Mudroom,Wide hallways,Room-darkening shades,Host greets you,Washer / Dryer,Mini fridge,Wide entryway,Free parking on premises,Cable TV,Cleaning before checkout,Patio or balcony,Internet,Hot tub,Baby monitor,Stair gates,Suitable for events,Washer,Pool with pool hoist,Flat path to guest entrance,Hair dryer,Children’s books and toys,No stairs or steps to enter,Hammock,Accessible-height bed,Keypad,Accessible-height toilet,Microwave,Private entrance,Beachfront,Netflix,Terrace,Other pet(s),Fixed grab bars for shower,Gas oven,Other,Ground floor access,Long term stays allowed,Lake access,Kitchenette,Hot water,Disabled parking spot,Building staff,Shower chair,Pocket wifi,Smoking allowed,Pets live on this property,Extra pillows and blankets,Crib,Well-lit path to entrance,Private living room,Buzzer/wireless intercom,DVD player,Firm mattress,Outlet covers,Roll-in shower,Paid parking off premises,Standing valet,TV,Table corner guards,Game console,Waterfront,Kitchen,Window guards,Mobile hoist,Balcony,Heated towel rack,Self check-in,Bathtub with bath chair,Hangers,Doorman,Toilet paper,Pack ’n Play/travel crib,Rain shower,Babysitter recommendations,Gym,Central air conditioning,Baby bath,En suite bathroom,High-resolution computer monitor,Heating,Warming drawer,Wide clearance to shower,Ski-in/Ski-out,Dishwasher,Wide entrance,Memory foam mattress,Printer,Cooking basics,Paid parking on premises,BBQ grill,Jetted tub,Bed linens,Smoke detector,Bedroom comforts,Body soap,Sound system,Private bathroom,Lock on bedroom door,Shared pool,Cat(s),Pillow-top mattress,Pool,Fireplace guards,Refrigerator,Handheld shower head,Steam oven,Outdoor seating,Laptop friendly workspace,Dog(s),Full kitchen,Air conditioning,Dishes and silverware,Ethernet connection,Touchless faucets,Fixed grab bars for toilet,Dryer,Carbon monoxide detector,Sun loungers,Day bed,Oven,Stove,Bath towel,Coffee maker,Bathroom essentials,Wheelchair accessible,toilet,Single level home,Exercise equipment,First aid kit,Amazon Echo,Wide doorway to guest bathroom,Formal dining area,Beach essentials,Hot water kettle,Family/kid friendly,Elevator,Free street parking,Iron,Wifi,Changing table,Garden or backyard,Walk-in shower,Espresso machine,Luggage dropoff allowed,Projector and screen,Ceiling fan
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,

In [16]:
lis.drop('amenities', axis=1, inplace=True)

In [17]:
lis=pd.concat([lis, amenities], axis=1)


In [18]:
lis.head()

,id,host_id,host_response_time,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,require_guest_profile_picture,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,x,y,z,property_type_Apartment,property_type_Boutique hotel,property_type_Camper/RV,property_type_Casa particular (Cuba),property_type_Chalet,property_type_Condominium,property_type_Dome house,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_Hotel,property_type_House,property_type_Loft,property_type_Other,property_type_Tiny house,property_type_Townhouse,property_type_Villa,room_type_Hotel room,room_type_Private room,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,Breakfast table,Indoor fireplace,Convection oven,Soaking tub,Extra space around bed,Essentials,Pets allowed,Electric profiling bed,Heated floors,Bidet,Beach view,Breakfast,Safety card,Fire extinguisher,24-hour check-in,Smart lock,HBO GO,Shampoo,EV charger,Wide entrance for guests,Children’s dinnerware,Lockbox,Smart TV,Wine cooler,High chair,Bathtub,Double oven,Mudroom,Wide hallways,Room-darkening shades,Host greets you,Washer / Dryer,Mini fridge,Wide entryway,Free parking on premises,Cable TV,Cleaning before checkout,Patio or balcony,Internet,Hot tub,Baby monitor,Stair gates,Suitable for events,Washer,Pool with pool hoist,Flat path to guest entrance,Hair dryer,Children’s books and toys,No stairs or steps to enter,Hammock,Accessible-height bed,Keypad,Accessible-height toilet,Microwave,Private entrance,Beachfront,Netflix,Terrace,Other pet(s),Fixed grab bars for shower,Gas oven,Other,Ground floor access,Long term stays allowed,Lake access,Kitchenette,Hot water,Disabled parking spot,Building staff,Shower chair,Pocket wifi,Smoking allowed,Pets live on this property,Extra pillows and blankets,Crib,Well-lit path to entrance,Private living room,Buzzer/wireless intercom,DVD player,Firm mattress,Outlet covers,Roll-in shower,Paid parking off premises,Standing valet,TV,Table corner guards,Game console,Waterfront,Kitchen,Window guards,Mobile hoist,Balcony,Heated towel rack,Self check-in,Bathtub with bath chair,Hangers,Doorman,Toilet paper,Pack ’n Play/travel crib,Rain shower,Babysitter recommendations,Gym,Central air conditioning,Baby bath,En suite bathroom,High-resolution computer monitor,Heating,Warming drawer,Wide clearance to shower,Ski-in/Ski-out,Dishwasher,Wide entrance,Memory foam mattress,Printer,Cooking basics,Paid parking on premises,BBQ grill,Jetted tub,Bed linens,Smoke detector,Bedroom comforts,Body soap,Sound system,Private bathroom,Lock on bedroom door,Shared pool,Cat(s),Pillow-top mattress,Pool,Fireplace guards,Refrigerator,Handheld shower head,Steam oven,Outdoor seating,Laptop friendly workspace,Dog(s),Full kitchen,Air conditioning,Dishes and silverware,Ethernet connection,Touchless faucets,Fixed grab bars for toilet,Dryer,Carbon monoxide detector,Sun loungers,Day bed,Oven,Stove,Bath towel,Coffee maker,Bathroom essentials,Wheelchair accessible,toilet,Single level home,Exercise equipment,First aid kit,Amazon Echo,Wide doorway to guest bathroom,Formal dining area,Beach essentials,Hot water kettle,Family/kid friendly,Elevator,Free street parking,Iron,Wifi,Changing table,Garden or backyard,Walk-in shower,Espresso machine,Luggage dropoff allowed,Projector and screen,Ceiling fan
0,6369,13660,within a few hours,0,1,0,Hispanoamérica

In [19]:
lis.to_csv('../madrid/listings_detailed/listings_visual.csv', index=False)